# DS/CMPSC 410 Fall 2024
## Instructor: Professor John Yen
## TA: Jin Peng and Al Lawati, Ali Hussain Mohsi

## Lab 7 Persist and Performance 
## The goals of this lab are for you to be able to
## - Choose the RDD for applying persist.
## - Apply the obove to compute hastag counts of Tweets after Boston Marathon Bombing (April 19 and April 20).
##  This lab includes four data sets, as explained below.
## Data
- The first dataset contains Boston Marathon Bombing collected on 4/19/2013.
- The second dataset contains Boston Marathon Bombing collected on 4/20/2013.
- To facilitate this analytics task both in the local mode and in the cluster mode, a smaller sampled dataset for each day's tweets has also been provided: sampled_4_19_tweets.csv and sampled_4_20_tweets.csv.  You should use these two data sets for running Spark in the local mode (using Jupyter Notebook).  However, you should change input data for spark-submit (cluster mode) to the big dataset for each day: BMB_4_19_tweets.csv and BMB_4_20_tweets.csv.
- You should create a Lab7 subdirectory under work, because more space is available under work directory. Download both datasets to your directory `<home>/work/Lab7/`.
- You should also download the Jupyter Notebook for Lab 7 to the same directory.
## Problem
- The problem we want to solve is to (1) find hashtags that in 4/19/2013 tweets and in 4/20/2013 tweets, and (2) calculate the difference of total hashtag counts of these hashtags (considering both days). You should save these hashtags together with their counts in a text file.
## Two Step
- You will first run in Spark local mode (and Jupyter Notebook) for the small sample for each day.
- After you obtain the result for small datasets, you can then convert the code for local mode into code for cluster mode, and submit the code to ICDS cluster and obtain run-time performance.  
1. Export the Jupyter Notebook as Lab7.py file.
2. Remove ``master="local"," from SparkContext.
3. Modify the Lab7.py file so that it reads from BMB_4_19_tweets.csv and BMB_4_20_tweets.csv.
4. Modify Lab7.py so that outputs are saved in directories different from those used in the local model
5. Follow step 4 and step 5 of the instructions in "Running Spark in Cluster Mode v3 using salloc" (Lab 5). 

## Submit the following items for Lab 7
- Completed Jupyter Notebook of Lab 7 (15%)
- Lab4C.py (used for spark-submit) (15%)
- Log file for Lab7.py             (15%)
- The output file of hashtag count difference generated by running spark-submit (cluster mode) on Lab7.py. (15%)
- Lab7P.py (adding persist() to RDD's of your choice) (15%)
- Log file for Lab7P.py  (15%)

## Total Points: 90 points

# Due: midnight, Oct 13th (Sunday).
# Early Submission Bonus (10 points): midnight Friday (Oct 11th)

In [ ]:
import pyspark

In [ ]:
from pyspark import SparkContext

## Because we are not using DataFrame in this lab, we will be creating SparkContext rather than SparkSession

- Note: We use "local" as the master parameter for ``SparkContext`` in this notebook so that we can run and debug it in ICDS Jupyter Server.  However, we need to remove ``"master="local",``later when you convert this notebook into a .py file for running it in the cluster mode.

In [ ]:
sc=SparkContext(master="local", appName="Lab7 Persist and BMB hastag changes")
sc

In [ ]:
sc.setLogLevel("WARN")

# Exercise 1 (5 points)  Add your name below 
## Answer for Exercise 1
- Your Name:

## Computing hastag count difference includes three steps:
- Step 1: Compute (and save) hashtag counts for April 19th tweets. 
- Step 2: Compute (and save) hashtag counts for April 20th tweets. 
- Step 3: Combine the hashtag counts of two days, compute their difference, sort based on the difference. Save the hashtag count difference. 

# Step 1 Compute (and save) hashtag counts for April 19th tweets.

## Complete the path and run the code below to read the file "sampled_4_19_BMBtweets.csv" from your Lab7 directory.

In [ ]:
tweets_D1_RDD = sc.textFile("/storage/home/juy1/work/Lab7/sampled_4_19_BMBtweets.csv")
tweets_D1_RDD


## Execute the code below, which computes the total count of hashtags in the input tweets, sort them by count (in descending order), and save them in an output directory:
- (a) Uses flatMap to "flatten" the list of tokens from each tweet (using split function) into a very large list of tokens.
- (b) Filter the token for hashtags.
- (c) Count the total number of hashtags in a way similar to Lab 2.
- (d) Sort the hashtag count in descending order.
- (e) Save the sorted hashtag counts in an output directory.

In [ ]:
tokens_D1_RDD = tweets_D1_RDD.flatMap(lambda line: line.strip().split(" "))

In [ ]:
hashtag_D1_RDD = tokens_D1_RDD.filter(lambda x: x.startswith("#"))

In [ ]:
hashtag_1_D1_RDD = hashtag_D1_RDD.map(lambda x: (x, 1))

In [ ]:
hashtag_count_D1_RDD = hashtag_1_D1_RDD.reduceByKey(lambda x, y: x+y , 5 )

In [ ]:
sorted_hashtag_count_D1_RDD = hashtag_count_D1_RDD.sortBy(lambda pair: pair[1] , ascending=False)

### Note: You need to complete the path with your output directory. 
### Note: You also need to change the directory names (e.g., replace "_local.txt" with "_cluster.txt") in your .py file for running Spark in cluster mode.

In [ ]:
output_path1 = "/storage/home/juy1/work/Lab7/sorted_BMB_hashtag_count_4_19_local.txt" 
sorted_hashtag_count_D1_RDD.saveAsTextFile(output_path1)

# Step 2 Compute and save hashtag counts for April 20th tweets.

In [ ]:
tweets_D2_RDD = sc.textFile("/storage/home/juy1/work/Lab7/sampled_4_20_BMBtweets.csv")
tweets_D2_RDD

## The code below computes the total count of hashtags in the input tweets, sort them by count (in descending order), and save them in an output directory:
- (a) Uses flatMap to "flatten" the list of tokens from each tweet (using split function) into a very large list of tokens.
- (b) Filter the token for hashtags.
- (c) Count the total number of hashtags in a way similar to Lab 2.
- (d) Sort the hashtag count in descending order.
- (e) Save the sorted hashtag counts in an output directory.

In [ ]:
tokens_D2_RDD = tweets_D2_RDD.flatMap(lambda line: line.strip().split(" "))

In [ ]:
hashtag_D2_RDD = tokens_D2_RDD.filter(lambda x: x.startswith("#"))

In [ ]:
hashtag_1_D2_RDD = hashtag_D2_RDD.map(lambda x: (x, 1) )

In [ ]:
hashtag_count_D2_RDD = hashtag_1_D2_RDD.reduceByKey(lambda x, y : x+y, 5 )

In [ ]:
sorted_hashtag_count_D2_RDD = hashtag_count_D2_RDD.sortBy(lambda pair: pair[1], ascending=False)

### Note: You need to complete the path with your output directory. 
### Note: You also need to change the directory names (e.g., replace "_sampled" with "_cluster") before you convert this notebook into a .py file for submiting it to ICDS cluster.  

In [ ]:
output_path2 = "/storage/home/juy1/work/Lab7/sorted_BMB_hashtag_count_4_20_local.txt" 
sorted_hashtag_count_D2_RDD.saveAsTextFile(output_path2)

# Step 3 Combine the hashcount of two days, compute their difference, and save sorted difference of hashtag counts.

## The code below joins the two hash_count key value pairs RDDs on hashtag (i.e., key), and  compute the difference of the hashtag counts between the two days.

In [ ]:
hashtag_count_D1_RDD.take(3)

In [ ]:
hashtag_count_D2_RDD.take(3)

In [ ]:
joined_hashtag_count_RDD = hashtag_count_D1_RDD.fullOuterJoin(hashtag_count_D2_RDD)

In [ ]:
joined_hashtag_count_RDD.take(5)

In [ ]:
none1_RDD = joined_hashtag_count_RDD.filter(lambda pair: pair[1][0]==None )

In [ ]:
none1_RDD.take(5)

In [ ]:
def tran_none(x):
    if (x==None) :
        return(0)
    else:
        return(x)

In [ ]:
hashtag_count_diff_RDD = joined_hashtag_count_RDD.map(lambda x: (x[0], tran_none(x[1][1])-tran_none(x[1][0])))

In [ ]:
hashtag_count_diff_RDD.take(5)

In [ ]:
sorted_hashtag_count_diff_RDD = hashtag_count_diff_RDD.sortBy(lambda x: x[1], ascending = False)

In [ ]:
sorted_hashtag_count_diff_RDD.take(5)

In [ ]:
output_path3 = "/storage/home/juy1/work/Lab7/sorted_BMB_hashtag_diff_local.txt" 
sorted_hashtag_count_diff_RDD.saveAsTextFile(output_path3)

In [ ]:
sc.stop()